In [105]:
import os
import numpy as np
import pickle
import random
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import math
import copy
from sklearn.utils import shuffle
from textblob import TextBlob
import pandas as pd
import re
import emoji as emoji

data= pd.read_csv("Tweets.csv")

# Process Data

In [106]:
# version 1 -> only remove @ and http:// 
def sub_remove_v1(data):
    val = re.sub('@', "",data["text"]) 
    val = re.sub(r" http\S+","",val) # remove http://
    return val
data['remove_1'] = data.apply(sub_remove_v1, axis=1) 


In [107]:
# Version 2 -> # remove anything starts with @ and http://
def sub_remove_v2(data):
    val = re.sub(r"@\S+", "",data["text"])
    val = re.sub(r" http\S+","",val)
    return val
data['remove_2'] = data.apply(sub_remove_v2, axis=1) 

In [108]:
# version 3 -> remove @, #, http:// 
# with removing #hash, we now treat hashtag has an indiviaul word. this version is to explore if the hashtag really helps or not 
def sub_remove_v3(data):
    val = re.sub('@', "",data["text"]) 
    val = re.sub(r"http\S+","",val) # remove http://
    val = re.sub('#', "",val) 
    return val
data['remove_3'] = data.apply(sub_remove_v3, axis=1) 


In [109]:
# version 4 -> collect hashtag 
#(i want to use this an innvatoive way -> with this, to just simply count the # of time in positive nad negative. 
#however, it needs to be well balanced.)
def sub_remove_v4(data):
    val = re.findall(r'#\S+',data["text"])
    return ' '.join(val) 

data['remove_4'] = data.apply(sub_remove_v4, axis=1) 
# I think this may use for additional features.. can we have BOW and also some other features at the same time? 

In [110]:
#version 5 -> covert emoji @ and http 
def sub_remove_v5(data):
    file = data['text']
    words = file.split(' ')
    for i in range(len(words)):
        try:
            words[i] = emoji.demojize(words[i])
            if words[i][:-1] in emoji.EMOJI_UNICODE.keys():
                words[i] = re.sub(r"_"," ",re.sub(r" ","",re.sub(r":","",words[i])[:-1]))
            else:
                pass
        except:
            pass
    
    val =  ' '.join(words)
    val = re.sub('@', "",val) 
    val = re.sub(r"http\S+","",val)
    return val 

            
data['remove_5'] = data.apply(sub_remove_v5, axis=1) 


In [111]:
#version 6 -> 1) emoji 2) remove @ 3) remvoe http 4) remove # and repeat hashtage 
def sub_remove_v6(data,hashtag_weight=1,remove_hash = 1):
    file = data['text']
    words = file.split(' ')
    for i in range(len(words)):
        try:
            words[i] = emoji.demojize(words[i])
            if words[i][:-1] in emoji.EMOJI_UNICODE.keys():
                words[i] = re.sub(r"_"," ",re.sub(r" ","",re.sub(r":","",words[i])[:-1]))
            else:
                pass
        except:
            pass
    
    val =  ' '.join(words)
    val = re.sub('@', "",val) 
    val = re.sub(r"http:\S+","",val)
      
    hashtag = re.findall(r'#\S+',val)
    length = len(hashtag)
    if length>0:
        output = ' '.join(hashtag * hashtag_weight)
        result = val + ' '+ output 
        
    else:
        result = val 
    if remove_hash == 1:
        
        result =re.sub(r"#","",result)
    else: 
        result 
    
    return result 

            
data['remove_6'] = data.apply(sub_remove_v6,args = (4,1) ,axis=1) # 4-> means total 5 times # 1 means remove #



In [117]:
data

tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
5      570300767074181121          negative                        1.0000   
6      570300616901320704          positive                        0.6745   
7      570300248553349120           neutral                        0.6340   
8      570299953286942721          positive                        0.6559   
9      570295459631263746          positive                        1.0000   
10     570294189143031808           neutral                        0.6769   
11     570289724453216256          positive                        1.0000   
12     570289584061480960          positive                        1.0000   
13     570287408438120448          positive                        0.6451   
14     570285904809598977          positive                        1.0000   
15     570282469121007616          negative                        0.6842   
16     570277724385734656          positive                        1.0000   
17     570276917301137409          negative                        1.0000   
18     570270684619923457          positive                        1.0000   
19     570267956648792064          positive                        1.0000   
20     570265883513384960          negative                        0.6705   
21     570264145116819457          positive                        1.0000   
22     570259420287868928          positive                        1.0000   
23     570258822297579520           neutral                        1.0000   
24     570256553502068736          negative                        1.0000   
25     570249102404923392          negative                        1.0000   
26     570239632807370753          negative                        1.0000   
27     570217831557677057           neutral                        0.6854   
28     570207886493782019          negative                        1.0000   
29     570124596180955136           neutral                        0.6150   
...                   ...               ...                           ...   
14610  569591765793165312          negative                        1.0000   
14611  569591730506371072           neutral                        1.0000   
14612  569591700416393216          negative                        1.0000   
14613  569591653121597440          negative                        1.0000   
14614  569591540944756737          negative                        1.0000   
14615  569591533617307648          negative                        1.0000   
14616  569591393540288512          negative                        1.0000   
14617  569591285150908416          positive                        1.0000   
14618  569591136534319105          negative                        1.0000   
14619  569590988395708416          positive                        1.0000   
14620  569590965880532993          negative                        1.0000   
14621  569590892085915649          negative                        1.0000   
14622  569590191758962688          negative                        1.0000   
14623  569590013278756865          positive                        0.6274   
14624  569589959088173056          negative                        1.0000   
14625  569589643487928321          positive                        1.0000   
14626  569589460226183168          negative                        1.0000   
14627  569588816438169600          negative                        1.0000   
14628  569588651925098496          positive                        1.0000   
14629  569588591602458624          negative                        1.0000   
14630  569588